In [60]:
import pandas as pd
import mygene
import numpy as np
import cobra

In [180]:
MMRN = cobra.io.read_sbml_model('data/models/sbml/MMRN.xml')
#CD = cobra.io.read_sbml_model('data/models/sbml/MMRNHep-CD.xml')
#WD = cobra.io.read_sbml_model('data/models/sbml/MMRNHep-WD.xml')

In [11]:
MMRN_rxns = pd.read_excel('data/models/xlsx/MMRN.xlsx',sheet_name='RXNS',index_col='ID')
MMRN_mets = pd.read_excel('data/models/xlsx/MMRN.xlsx',sheet_name='METS',index_col='ID')

rxn_map = pd.read_csv('../../../../Integration/scripts/Human-GEM/model/reactions.tsv',sep='\t',index_col='rxns')
met_map = pd.read_csv('../../../../Integration/scripts/Human-GEM/model/metabolites.tsv',sep='\t',index_col='mets')

Human1_rxns = pd.read_excel('../../../../Integration/scripts/Human-GEM/model/Human-GEM.xlsx',
                            sheet_name='RXNS',index_col='ID')

Human1_mets = pd.read_excel('../../../../Integration/scripts/Human-GEM/model/Human-GEM.xlsx',
                            sheet_name='METS',index_col='REPLACEMENT ID')

# REACTIONS

In [80]:
MMRN_rxns = pd.read_excel('data/models/xlsx/MMRN.xlsx',sheet_name='RXNS',index_col='ID')
Human1_rxns = pd.read_excel('../../../../Integration/scripts/Human-GEM/model/Human-GEM.xlsx',
                            sheet_name='RXNS',index_col='ID')
rxn_map = pd.read_csv('../../../../Integration/scripts/Human-GEM/model/reactions.tsv',sep='\t',index_col='rxns')

In [81]:
toadd = ['rxnKEGGID','rxnMetaNetXID','rxnREACTOMEID','rxnRheaID','rxnRheaMasterID','rxnBiGGID']
done = []
for search in ['rxnRetired','rxnRecon3DID','rxnHMR2ID']:
    rxn_map2 = rxn_map
    print(search)
    if search == 'rxnBiGGID':
        rxn_map2['rxns'] = rxn_map2.index
        rxn_map2['BiGGID'] = rxn_map2['rxnBiGGID']
        rxn_map2 = rxn_map2.set_index('BiGGID')
    else:
        rxn_map2['rxns'] = rxn_map2.index
        rxn_map2 = rxn_map.set_index(search)
        rxn_map2 = rxn_map2.astype('str')
    rxn_map2 = rxn_map2.drop_duplicates()
    for index,row in MMRN_rxns.iterrows():
        if index in list(rxn_map2.index) and index not in done:
            for col in toadd:
                val = str(rxn_map2.at[index,col])
                MMRN_rxns.at[index,col] = str(val)
            done.append(index)
            
            try:
                human = rxn_map2.at[index,'rxns']
                miriam = Human1_rxns.at[human,'MIRIAM']
                MMRN_rxns.at[index,'MIRIAM'] = miriam
            except TypeError:
                pass

rxnRetired
rxnRecon3DID
rxnHMR2ID


In [82]:
for index,row in MMRN_rxns.iterrows():
    if 'RHEA' in str(row['rxnRheaID']):
        rhea = str(row['rxnRheaID']).replace('RHEA:','') + '; ' +str(row['rxnRheaMasterID']).replace('RHEA:','')
        MMRN_rxns.at[index,'rxnRheaFinal'] = rhea

In [83]:
for index,row in MMRN_rxns.iterrows():
    miriam = str(row['MIRIAM'])
    parts = miriam.split(';')
    for part in parts:
        if 'SBO' in part:
            MMRN_rxns.at[index,'rxnSBOTerms'] = part.replace('sbo/','')

In [84]:
MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace('EC:','')
MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace(',',';')
MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace(' ',';')
MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace('.-','')
MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace(';or','')
for index,row in MMRN_rxns.iterrows():
    if 'TCDB' in str(row['EC-NUMBER']):
        MMRN_rxns.at[index,'EC-NUMBER'] = ''

/var/folders/zc/h_66z5bx601_njjc__92xndnxk18rv/T/ipykernel_54170/2896705547.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  MMRN_rxns['EC-NUMBER'] = MMRN_rxns['EC-NUMBER'].str.replace('.-','')


In [85]:
MMRN_rxns['NOTE'] = MMRN_rxns['NOTE'].str.replace('PROTEIN_CLASS: ','')
MMRN_rxns['NOTE'] = MMRN_rxns['NOTE'].str.replace('EC:','')
MMRN_rxns['NOTE'] = MMRN_rxns['NOTE'].str.replace('.-','')
for index,row in MMRN_rxns.iterrows():
    if 'TCDB' in str(row['NOTE']):
        MMRN_rxns.at[index,'NOTE'] = ''
    if 'Included' in str(row['NOTE']):
        MMRN_rxns.at[index,'NOTE'] = ''
    if 'Spontaneous' in str(row['NOTE']) or 'spontaneous' in str(row['NOTE']):
        MMRN_rxns.at[index,'NOTE'] = ''
    if 'Non' in str(row['NOTE']):
        MMRN_rxns.at[index,'NOTE'] = ''

/var/folders/zc/h_66z5bx601_njjc__92xndnxk18rv/T/ipykernel_54170/3689479095.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  MMRN_rxns['NOTE'] = MMRN_rxns['NOTE'].str.replace('.-','')


In [86]:
MMRN_rxns['ECNumber'] = MMRN_rxns[["EC-NUMBER","NOTE"]].apply(lambda row: ';'.join(row.values.astype(str)), axis=1)
MMRN_rxns['ECNumber'] = MMRN_rxns['ECNumber'].str.replace('nan;','')
MMRN_rxns['ECNumber'] = MMRN_rxns['ECNumber'].str.replace(';nan','')
MMRN_rxns['ECNumber'] = MMRN_rxns['ECNumber'].str.replace(';','; ')
for index,row in MMRN_rxns.iterrows():
    print(index)
    ec = str(row['ECNumber']).split('; ')
    ec = list(filter(None, ec))
    ec = str('; '.join(ec))
    MMRN_rxns.at[index,'ECNumber'] = ec

HMR_0154
HMR_0189
HMR_0193
HMR_0197
HMR_0201
HMR_0206
HMR_0210
HMR_0214
HMR_0223
HMR_0230
HMR_0234
HMR_0238
HMR_0242
HMR_0246
HMR_0250
HMR_0256
HMR_0260
HMR_0264
HMR_0268
HMR_0272
HMR_0276
HMR_0280
HMR_0284
HMR_0290
HMR_0294
HMR_0298
HMR_0302
HMR_0306
HMR_0310
HMR_0314
HMR_0320
HMR_0324
HMR_0328
HMR_0332
HMR_0338
HMR_0342
HMR_0346
HMR_0350
HMR_0354
HMR_0358
HMR_0362
HMR_0366
HMR_0370
HMR_0374
HMR_0378
HMR_0382
HMR_0386
HMR_0390
HMR_0394
HMR_0398
HMR_0402
HMR_0406
HMR_0410
HMR_0414
HMR_0418
HMR_0422
HMR_0426
HMR_0430
HMR_0434
HMR_0438
HMR_3009
HMR_3475
HMR_3476
HMR_3477
HMR_0449
AGTim
AGTix
ARGSS
ASNNm
ASNS1
ASPTAm
DASPO1p
NACASPAH
RE2031M
ALAR
ASPTA
r0127
ARGSL
HMR_9897
HMR_9898
HMR_9899
HMR_9900
HMR_3802
HMR_3804
HMR_3811
HMR_3831
HMR_3862
HMR_3865
HMR_3873
HMR_8654
HMR_3890
HMR_3892
HMR_9802
HMR_3899
HMR_3903
HMR_4109
HMR_4114
HMR_4115
HMR_4172
HMR_4196
HMR_4197
HMR_4285
HMR_4690
HMR_6968
HMR_6970
HMR_9868
HMR_6971
HMR_9867
HMR_6972
HMR_7641
HMR_8626
COKECBESr
HMR_9895
HMR_3986
HMR_3

HMR_2715
HMR_2718
HMR_2722
HMR_2727
HMR_2731
HMR_2732
HMR_2734
HMR_2735
HMR_2737
HMR_2738
HMR_2740
HMR_2741
HMR_2744
HMR_2748
HMR_2750
HMR_2753
HMR_2754
HMR_2757
HMR_2758
HMR_2760
HMR_2764
HMR_2769
HMR_2770
HMR_2772
HMR_2773
HMR_2775
HMR_2776
HMR_3519
HMR_3520
HMR_8417
HMR_8418
HMR_3027
HMR_3028
HMR_3033
HMR_3034
HMR_3035
HMR_1440
HMR_1445
HMR_1448
HMR_1451
HMR_1457
HMR_1460
HMR_1465
HMR_1467
HMR_1470
HMR_1473
HMR_1477
HMR_1478
HMR_1479
HMR_1484
HMR_1490
HMR_1493
HMR_1494
HMR_1495
HMR_1496
HMR_1500
HMR_1502
HMR_1503
HMR_1504
HMR_1505
HMR_1509
HMR_1512
HMR_1516
HMR_1519
HMR_1526
HMR_1533
HMR_1558
HMR_1565
HMR_1535
HMR_1536
HMR_1538
HMR_1539
HMR_1540
HMR_1543
HMR_1544
HMR_1545
HMR_1546
HMR_1547
HMR_1548
HMR_1549
HMR_1550
HMR_1551
HMR_1552
ACACT1x
C14STRr
C3STDH1Pr
C3STDH1r
C3STKR2r
C4STMO1r
C4STMO2Pr
C4STMO2r
DHCR241r
DHCR242r
DHCR243r
DHCR71r
DHCR72r
DMATTx
DPMVDx
EBP1r
EBP2r
GRTTx
HMGCOARr
HMGCOASi
HMGCOASim
HMGLx
IPDDIx
LNS14DM
LNSTLSr
LSTO1r
LSTO2r
MEVK1x
SOAT11
SOAT11r
SOAT12
SOAT12

FAOXC16080x
FAOXC161802m
FAOXC16180m
FAOXC170m
FAOXC180
FAOXC180x
FAOXC1811601m
FAOXC1811602m
FAOXC1811603m
FAOXC182806m
FAOXC18280m
FAOXC183803m
FAOXC183806m
FAOXC183806x
FAOXC18480m
FAOXC18480x
FAOXC200180m
FAOXC200180x
FAOXC2031836m
FAOXC204
FAOXC204184m
FAOXC2051843m
FAOXC2051843x
FAOXC2242046m
FAOXC2242046x
FAOXC2251836m
FAOXC2251836x
FAOXC2252053m
FAOXC2252053x
FAOXC226
FAOXC226205m
FAOXC226205x
FAOXC240200x
FAOXC241181x
FAOXC2442246x
FAOXC2452253x
FAOXC2452256x
FAOXC246226x
FAOXC260240x
FAOXC80
FDH
HEXCCPT1
HEXCCPT2
HEXCCRNt
HPDCACRNCPT1
HPDCACRNt
LGNCCRNt
LNELDCCPT2
LNELDCCRNt
LNLCCPT1
LNLCCPT2
LNLCCRNt
LNLNCACPT1
LNLNCACPT2
LNLNCACRNt
LNLNCGCPT1
LNLNCGCRNt
MCD
MCDm
MCDp
MCOATAm
NRVNCCPT2
NRVNCCRNt
PCRNtm
PHYHx
PTDCACRNCPT1
PTDCACRNt
PTE2x
PTE3x
PTE4x
PTE5x
STRDNCCPT2
STRDNCCRNt
TETPENT3CPT2
TETPENT3CRNt
TETPENT6CPT2
TETPENT6CRNt
TETTET6CPT1
TETTET6CPT2
TETTET6CRNt
TMNDNCCPT2
TMNDNCCRNt
TTDCPT2
TTDCRNt
r0308
r0309
r0310
r0311
r0430
r0431
r0432
r0433
r0434
r0437
r0438
r0440
r044

ALKP
CHAT
CHATn
CHLPCTD
CHOLK
DAGKn_hs
ETHAK
GLYKm
PETHCT
PI4P5K
r0186
r0245
RE2977C
GPDDACHOL
G3PD1
GLYK
HMR_0448
HMR_0450
HMR_0478
HMR_0479
HMR_0481
HMR_0483
HMR_0482
HMR_0484
HMR_0635
HMR_0636
HMR_0638
HMR_0641
HMR_0646
HMR_0647
HMR_0648
HMR_0649
HMR_0651
HMR_4627
HMR_4838
HMR_4839
HMR_7611
HMR_7612
HMR_7613
HMR_7755
HMR_8421
HMR_8424
HMR_9805
HMR_0457
HMR_0460
HMR_3750
HMR_3752
HMR_3770
HMR_3771
HMR_3772
HMR_3782
HMR_3794
HMR_3841
HMR_3843
HMR_3849
HMR_3852
HMR_3860
HMR_3883
HMR_3901
HMR_3974
HMR_4198
HMR_4467
HMR_4199
HMR_4284
HMR_4348
HMR_4466
HMR_4584
HMR_4696
HMR_4700
HMR_4742
HMR_4788
HMR_4789
HMR_4791
HMR_6409
HMR_7702
HMR_7703
HMR_8433
HMR_8434
HMR_8435
HMR_8436
HMR_8437
HMR_8439
HMR_8440
HMR_8441
HMR_8442
HMR_9718
HMR_4937
2AMACHYD
AACTOOR
ALASm
BHMT
GHMT2rm
GLYATm
GNMT
PGCD
SERHL
SPTix
r0160
r0552
r0553
RE2111M
RE2117M
RE2427M
RE2428M
RE2429M
GHMT2r
PSERT
PSP_L
THRS
THRD_L
PPIOGLYc
TIGGLYc
HEXGLYc
SUBGLYc
HMR_4394
HMR_4381
HMR_4379
HMR_4301
HMR_4377
HMR_4375
HMR_4391
HMR_4

HMR_2470
HMR_6398
HMR_6399
HMR_6401
HMR_6402
HMR_6404
2AMADPTm
2OXOADOXm
AATAi
GHMT3m
LPCOXp
SACCD3m
SACCD4m
r0450
r0525
r0594
HMR_4239
HMR_4241
HMR_4288
HMR_4599
HMR_4667
HMR_4737
HMR_4739
HMR_4740
HMR_6415
HMR_6979
HMR_6980
HMR_6981
HMR_6982
HMR_6984
HMR_6985
HMR_8017
HMR_8018
HMR_8019
HMR_8021
HMR_9832
HMR_9833
HMR_9834
HMR_6992
HMR_6993
HMR_6994
HMR_6995
HMR_6996
HMR_6997
HMR_6998
HMR_6999
HMR_7000
HMR_7001
HMR_7002
HMR_7003
HMR_7004
HMR_7005
HMR_7006
HMR_7007
HMR_7008
HMR_7011
HMR_7012
HMR_7013
HMR_7014
HMR_7015
HMR_7016
HMR_7017
HMR_7018
HMR_7019
HMR_7020
HMR_7021
HMR_7022
HMR_7023
HMR_7024
HMR_7025
HMR_7026
HMR_7027
HMR_7028
HMR_7029
HMR_7030
HMR_7031
HMR_7033
HMR_7034
HMR_7035
HMR_7036
HMR_7037
HMR_7038
HMR_7039
HMR_7040
HMR_7041
HMR_7042
HMR_7043
HMR_7044
HMR_7045
HMR_7046
HMR_7047
HMR_7048
HMR_7049
HMR_7050
HMR_7051
HMR_7052
HMR_7053
HMR_7054
HMR_7055
HMR_7056
HMR_7057
HMR_7058
HMR_7059
HMR_7060
HMR_7061
HMR_7063
HMR_7064
HMR_7065
HMR_7066
HMR_7068
HMR_7069
HMR_7071
HMR_7073


HMR_6792
HMR_6798
HMR_6799
HMR_6800
HMR_6801
HMR_6802
HMR_6803
HMR_6806
HMR_6807
HMR_6811
HMR_6814
HMR_6815
HMR_6817
HMR_6818
HMR_6819
HMR_6820
HMR_6822
HMR_6823
HMR_6824
HMR_6826
HMR_6827
HMR_6828
HMR_6829
HMR_6830
HMR_6831
HMR_6834
HMR_6835
HMR_6836
HMR_6837
HMR_6838
HMR_6839
HMR_6840
HMR_6841
HMR_6842
HMR_6843
HMR_6844
HMR_6845
HMR_6848
HMR_6849
HMR_6850
HMR_6851
HMR_6854
HMR_6855
HMR_6874
HMR_6875
HMR_6876
HMR_6877
HMR_6878
HMR_6879
HMR_6880
HMR_6881
HMR_6882
HMR_6883
HMR_6885
HMR_6886
HMR_6888
HMR_6889
HMR_7628
HMR_7756
HMR_8794
HMR_8795
HMR_8796
GPIAT
GPIDA2er
GPIDAer
M4ATAer
RE1447M
RE1957G
RE2972M
RE2974G
RE2974R
RE3269C
RE3273R
HMR_4657
HMR_4744
HMR_4746
HMR_4748
HMR_4750
HMR_4752
HMR_4755
HMR_4757
HMR_4758
HMR_4762
HMR_9717
HMR_4763
HMR_4764
HMR_8634
HMR_4768
HMR_4769
HMR_4770
HMR_4771
HMR_4772
HMR_4773
HMR_6395
ACCOALm
ACS2
MMSAD3m
PPCOAOm
r1154
ACCOAL
2M3OVCOAm
HMR_0153
HMR_3797
HMR_3800
HMR_4105
HMR_4106
HMR_4107
HMR_4282
HMR_4331
HMR_3796
HMR_4459
HMR_4460
HMR_4464
HMR_80

HMR_2777
HMR_9200
HMR_7789
HMR_7791
HMR_7929
HMR_7933
HMR_7944
HMR_7947
HMR_7949
HMR_7953
HMR_7956
HMR_7960
HMR_7963
HMR_7966
HMR_7969
HMR_7981
HMR_7983
HMR_7985
HMR_7988
HMR_7989
HMR_8030
HMR_8058
HMR_8060
HMR_8234
HMR_8236
HMR_8239
HMR_8358
HMR_8382
HMR_8526
HMR_8553
HMR_8594
HMR_8651
HMR_8715
HMR_8716
HMR_8719
HMR_8722
HMR_8737
HMR_8851
HMR_8855
HMR_8859
HMR_8888
HMR_8906
HMR_8914
HMR_8915
HMR_8919
HMR_9177
HMR_9626
HMR_9628
HMR_9629
HMR_9630
HMR_9631
HMR_9632
HMR_9633
HMR_9634
HMR_9716
HMR_9732
HMR_8973
HMR_9835
HMR_9848
HMR_9876
HMR_9909
2HBt2
2HCO3_NAt
34DHPHEt
5ADTSTSTERONEGLCte
5FTHFt2
5HTRPVESSEC
5MTHFt
5MTHFt2
ABUTt4_2_r
ACETONEt2
ACHVESSEC
ADEt
ADNt
ADPRIBt
ADRNLPVESSEC
AHANDROSTANGLCte
AKGt4_3
ALAASNNaEx
ALACYSNaEx
ALADGLNexR
ALADGLYexR
ALAGLNexR
ALAGLNNaEx
ALAGLYexR
ALASERNaEx
ALAt4
ALATHRNaEx
ALAtN1
ANDRSTRNGLCte
ARACHDt2
ARGLYSex
ARGt4
ASCBt4
ASNALANaEx
ASNCYSNaEx
ASNGLNNaEx
ASNSERNaEx
ASNt4
ASNTHRNaEx
ASNtN1
ASPDt6
BALAVECSEC
BILDGLCURt
BILDGLCURte
BILGLCURt
BILGLCURte


HMR_6019
HMR_6021
HMR_6026
HMR_6029
HMR_6030
HMR_6033
HMR_6036
HMR_6038
HMR_6039
HMR_6041
HMR_6042
HMR_6043
HMR_6045
HMR_6047
HMR_6048
HMR_6049
HMR_6050
HMR_6052
HMR_6053
HMR_6057
HMR_6058
HMR_6059
HMR_6060
HMR_6061
HMR_6062
HMR_6066
HMR_6067
HMR_6095
HMR_6099
HMR_6100
HMR_6102
HMR_6103
HMR_6104
HMR_6105
HMR_6106
HMR_6107
HMR_6142
HMR_6143
HMR_6144
HMR_6214
HMR_6215
HMR_6216
HMR_6217
HMR_6218
HMR_6219
HMR_6220
HMR_6221
HMR_6222
HMR_6223
HMR_6224
HMR_6225
HMR_6226
HMR_6227
HMR_6228
HMR_6229
HMR_6230
HMR_6231
HMR_6232
HMR_6233
HMR_6234
HMR_6235
HMR_6236
HMR_6237
HMR_6238
HMR_6239
HMR_6240
HMR_6248
HMR_6249
HMR_6250
HMR_6251
HMR_6252
HMR_6253
HMR_6254
HMR_6255
HMR_6256
HMR_6257
HMR_6258
HMR_6259
HMR_6260
HMR_6261
HMR_6262
HMR_6263
HMR_6264
HMR_6265
HMR_6266
HMR_6267
HMR_6268
HMR_6269
HMR_6270
HMR_6271
HMR_6272
HMR_6273
HMR_6274
HMR_6351
HMR_6352
HMR_6353
HMR_6354
HMR_6355
HMR_6356
HMR_6357
HMR_6358
HMR_6359
HMR_6360
HMR_6361
HMR_6362
HMR_6363
HMR_6364
HMR_6365
HMR_6367
HMR_6369
HMR_6380
H

HMR_6431
HMR_6438
HMR_6446
HMR_6454
HMR_6469
HMR_6481
HMR_6513
HMR_6517
HMR_6522
HMR_6618
HMR_6620
HMR_6781
HMR_6890
HMR_6902
HMR_7638
HMR_7708
HMR_7758
HMR_7815
HMR_7816
HMR_7825
HMR_7826
HMR_7827
HMR_7828
HMR_7829
HMR_7830
HMR_7831
HMR_7832
HMR_7833
HMR_7834
HMR_7835
HMR_7836
HMR_7837
HMR_7838
HMR_7839
HMR_7840
HMR_7841
HMR_7842
HMR_7843
HMR_7844
HMR_7845
HMR_7846
HMR_7847
HMR_7848
HMR_7849
HMR_7850
HMR_7851
HMR_7852
HMR_7853
HMR_7854
HMR_7914
HMR_7917
HMR_7923
HMR_7995
HMR_7998
HMR_8000
HMR_8001
HMR_8009
HMR_3746
HMR_8093
HMR_8099
HMR_8120
HMR_8122
HMR_8124
HMR_8126
HMR_8128
HMR_8356
HMR_8365
HMR_8411
HMR_8438
HMR_8505
HMR_8510
HMR_8513
HMR_8532
HMR_8562
HMR_8627
HMR_8655
HMR_8657
HMR_8680
HMR_8731
HMR_8741
HMR_8742
HMR_8745
HMR_8747
HMR_8785
HMR_8848
HMR_8890
HMR_8891
HMR_9174
HMR_9194
HMR_9675
HMR_9676
HMR_9677
HMR_9870
HMR_0025
HMR_0030
HMR_0984
HMR_1089
HMR_1090
HMR_1094
HMR_1095
HMR_2564
HMR_4274
HMR_4275
HMR_4277
HMR_4535
HMR_4893
HMR_4904
HMR_4916
HMR_5422
HMR_5423
HMR_5424
H

EXC_BOTH_C06423[s]
EXC_BOTH_C01601[s]
EXC_BOTH_C01571[s]
EXC_BOTH_Cx1379[s]
EXC_BOTH_m74[s]
EXC_BOTH_C00001[s]
EXC_BOTH_C00041[s]
EXC_BOTH_C00152[s]
EXC_BOTH_C00064[s]
EXC_BOTH_C00082[s]
EXC_BOTH_C00097[s]
EXC_BOTH_C00062[s]
EXC_BOTH_C00037[s]
EXC_BOTH_C00148[s]
EXC_BOTH_C00065[s]
EXC_BOTH_C00049[s]
EXC_BOTH_C00025[s]
EXC_BOTH_C00009[s]
EXC_BOTH_C00014[s]
EXC_BOTH_C00059[s]
EXC_BOTH_C14818[s]
EXC_BOTH_C00288[s]
EXC_BOTH_C00080[s]
EXC_BOTH_C01330[s]
EXC_BOTH_C00238[s]
EXC_BOTH_C00114[s]
EXC_BOTH_C00189[s]
EXC_BOTH_C00116[s]
EXC_BOTH_C00033[s]
EXC_BOTH_C00077[s]
EXC_BOTH_C00094[s]
EXC_BOTH_m91[s]
EXC_BOTH_C05828[s]
EXC_BOTH_C00334[s]
EXC_BOTH_C00547[s]
EXC_BOTH_C00643[s]
EXC_BOTH_C00788[s]
EXC_BOTH_C14819[s]
EXC_BOTH_C00469[s]
EXC_BOTH_C03479[s]
EXC_BOTH_C00870[s]
EXC_BOTH_C00283[s]
EXC_BOTH_C02918[s]
EXC_BOTH_C01081[s]
EXC_BOTH_C03028[s]
EXC_BOTH_C00032[s]
EXC_BOTH_C00120[s]
EXC_BOTH_Cx1006[s]
EXC_BOTH_Cx1153[s]
EXC_BOTH_Cx1112[s]
EXC_BOTH_Cx1163[s]
EXC_BOTH_Cx1178[s]
EXC_BOTH_Cx829[s]


In [87]:
counter = 0
MMRN_index = []
MMRN_rxns['RetiredID'] = MMRN_rxns.index
for i in MMRN_rxns.index:
    counter += 1
    string_counter = str(counter)
    string_counter = string_counter.zfill(5)
    MMRN_index.append(str('MMRNR'+string_counter))
MMRN_rxns['MMRN_index'] = MMRN_index
MMRN_rxns = MMRN_rxns.set_index('MMRN_index')

In [88]:
MMRN_rxns.fillna('', inplace=True)
MMRN_rxns = MMRN_rxns.replace(np.nan, '', regex=True)

In [89]:
print(list(MMRN_rxns.columns))

['#', 'NAME', 'EQUATION', 'EC-NUMBER', 'GENE ASSOCIATION', 'LOWER BOUND', 'UPPER BOUND', 'OBJECTIVE', 'COMPARTMENT', 'MIRIAM', 'SUBSYSTEM', 'REPLACEMENT ID', 'NOTE', 'REFERENCE', 'CONFIDENCE SCORE', 'rxnKEGGID', 'rxnMetaNetXID', 'rxnREACTOMEID', 'rxnRheaID', 'rxnRheaMasterID', 'rxnBiGGID', 'rxnRheaFinal', 'rxnSBOTerms', 'ECNumber', 'RetiredID']


In [90]:
MMRN_rxns.to_csv('data/models/xlsx/MMRN_rxns.csv')

# METABOLITES

In [91]:
MMRN_mets = pd.read_excel('data/models/xlsx/MMRN.xlsx',sheet_name='METS',index_col='ID')
met_map = pd.read_csv('../../../../Integration/scripts/Human-GEM/model/metabolites.tsv',sep='\t',index_col='mets')
Human1_mets = pd.read_excel('../../../../Integration/scripts/Human-GEM/model/Human-GEM.xlsx',
                            sheet_name='METS',index_col='REPLACEMENT ID')

In [92]:
for index,row in MMRN_mets.iterrows():
    MMRN_mets.at[index,'old'] = row['REPLACEMENT ID'].split('[')[0]
    MMRN_mets.at[index,'RetiredID'] = row['REPLACEMENT ID']
    if '/' in str(row['MIRIAM']):
        parts = row['MIRIAM'].split(';')
        for part in parts:
            part = part.split('/')
            if 'CHEBI' in part[0]:
                MMRN_mets.at[index,part[0]] = 'CHEBI:'+part[1]
            else:
                MMRN_mets.at[index,part[0]] = part[1]

In [93]:
met_map['mets'] = met_map.index
met_map_kegg = met_map.set_index('metKEGGID')
met_map_kegg = met_map_kegg[~met_map_kegg.index.duplicated(keep='first')]
for index,row in MMRN_mets.iterrows():
    if row['old'] in met_map_kegg.index:
        kegg = MMRN_mets.at[index,'old']
        MMRN_mets.at[index,'kegg.compound'] = kegg
        MMRN_mets.at[index,'hmdb'] = met_map_kegg.at[kegg,'metHMDBID']
        MMRN_mets.at[index,'pubchem.compound'] = met_map_kegg.at[kegg,'metPubChemID']
        MMRN_mets.at[index,'obo.chebi:CHEBI'] = met_map_kegg.at[kegg,'metChEBIID']
        MMRN_mets.at[index,'lipidmaps'] = met_map_kegg.at[kegg,'metLipidMapsID']
        
        human = met_map_kegg.at[kegg,'mets']
        miriam = Human1_mets.at[human,'MIRIAM']
        MMRN_mets.at[index,'MIRIAM'] = miriam

In [94]:
for index,row in MMRN_mets.iterrows():
    miriam = str(row['MIRIAM'])
    parts = miriam.split(';')
    for part in parts:
        if 'SBO' in part:
            MMRN_mets.at[index,'metSBOTerms'] = part.replace('sbo/','')

In [95]:
counter = 0
done = {}
for index,row in MMRN_mets.iterrows():
    if row['NAME'] not in done:
        counter += 1
        string_counter = str(counter)
        string_counter = string_counter.zfill(4)
        new_id = 'MMRNM' + str(string_counter) + row['COMPARTMENT']
        MMRN_mets.at[index,'REPLACEMENT ID'] = new_id
        done[row['NAME']] = 'MMRNM' + str(string_counter)
    else:
        new_id = done[row['NAME']] + row['COMPARTMENT']
        MMRN_mets.at[index,'REPLACEMENT ID'] = new_id

In [96]:
MMRN_mets.to_csv('data/models/xlsx/MMRN_mets.csv')

# GENES

In [97]:
MMRN_gene = pd.read_excel('data/models/xlsx/MMRN.xlsx',sheet_name='GENES',index_col='NAME')

In [98]:
mg = mygene.MyGeneInfo()
ginfo = mg.querymany(list(MMRN_gene.index), scopes='ensembl.gene',fields='all')

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3466...done.
Finished.
57 input query terms found no hit:
	['ENSMUSG00000024952', 'ENSMUSG00000039530', 'ENSMUSG00000079834', 'ENSMUSG00000096994', 'ENSMUSG000
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [99]:
all_miriams = {}
for gene in ginfo:
    query = gene['query']
    try:
        ensembl = gene['ensembl']
        if isinstance(ensembl, list):
            ensembl_gene = []
            ensembl_trans = []
            ensembl_protein = []
            for i in ensembl:
                ensembl_gene.append(i['gene'])
                if isinstance(i['transcript'], list):
                    for j in i['transcript']:
                        ensembl_trans.append(j)
                else:
                    ensembl_trans.append(i['transcript'])
                if isinstance(i['protein'], list):
                    for j in i['protein']:
                        ensembl_protein.append(j)
                else:
                    ensembl_protein.append(i['protein'])
        else:
            ensembl_gene = ensembl['gene']
            ensembl_trans = ensembl['transcript']
            ensembl_protein = ensembl['protein']
        if isinstance(ensembl_gene, list):
            ensembl_gene = str('; '.join(ensembl_gene))
        if isinstance(ensembl_trans, list):
            ensembl_trans = str('; '.join(ensembl_trans))
        if isinstance(ensembl_protein, list):
            ensembl_protein = str('; '.join(ensembl_protein))
            
        MMRN_gene.at[query,'ensembl_gene'] = ensembl_gene
        MMRN_gene.at[query,'ensembl_transcript'] = ensembl_trans
        MMRN_gene.at[query,'ensembl_protein'] = ensembl_protein  
    except KeyError:
        pass

In [100]:
for gene in ginfo:
    query = gene['query']
    try:
        entrez = gene['entrezgene']
        MMRN_gene.at[query,'entrez_gene'] = entrez
    except KeyError:
        pass

In [101]:
for gene in ginfo:
    query = gene['query']
    try:
        refseq = gene['refseq']['genomic']
        MMRN_gene.at[query,'refseq_gene'] = refseq
    except KeyError:
        pass

In [102]:
for gene in ginfo:
    query = gene['query']
    try:
        uniprot = gene['uniprot']
        miriams = []
        for item in uniprot:
            if isinstance(uniprot[item], list):
                for j in uniprot[item]:
                    miriams.append(j)
            else:
                miriams.append(uniprot[item])
        MMRN_gene.at[query,'uniprot_gene'] = str(';'.join(miriams))
    except KeyError:
        pass

In [103]:
MMRN_gene.to_csv('data/models/xlsx/MMRN_genes.csv')

# MMRN COMPARISON WITH MMRN_CONSIST

In [171]:
MMRN_mets = pd.read_excel('data/models/xlsx/MMRN_revised.xlsx',sheet_name='METS')
MMRNHep_mets = pd.read_excel('data/models/xlsx/genericLiverLFD.xlsx',sheet_name='METS')

In [172]:
d = dict(zip(MMRN_mets['ID'],MMRN_mets['REPLACEMENT ID']))

In [173]:
for index,row in MMRNHep_mets.iterrows():
    if row['ID'] in d:
        MMRNHep_mets.at[index,'REPLACEMENT ID'] = d[row['ID']]
    else:
        print(row['ID'])  
MMRNHep_mets.to_csv('data/models/xlsx/MMRNHep_mets.csv')

biomass[x]
cofactor_pool_biomass[c]
protein_pool_biomass[c]
lipid_pool_biomass[c]
nucleotide_pool_biomass[c]
glycogen_pool_biomass[c]
metabolite_pool_biomass[c]


In [174]:
len(MMRNHep_mets)

3773

In [175]:
MMRN_rxns = pd.read_excel('data/models/xlsx/MMRN_revised.xlsx',sheet_name='RXNS')
MMRNHep_rxns = pd.read_excel('data/models/xlsx/genericLiverLFD.xlsx',sheet_name='RXNS')

In [176]:
d = dict(zip(MMRN_rxns['RetiredID'],MMRN_rxns['ID']))

In [177]:
for index,row in MMRNHep_rxns.iterrows():
    if row['ID'] in d:
        MMRNHep_rxns.at[index,'ID'] = d[row['ID']]
    else:
        print(row['ID'])  

MMRN_Lipid_pool
MMRN_Protein_pool
MMRN_Nucleotide_pool
MMRN_glycogen
MMRN_Biomass
HMR_biomass_Renalcancer(with ATP)
EXC_OUT_Biomass
EXC_OUT_Lipid_pool_biomass
EXC_OUT_Protein_pool_biomass
EXC_OUT_Nucleotide_pool_biomass
EXC_OUT_cofactors_vitamins
EXC_OUT_glycogen


In [178]:
MMRNHep_rxns.to_csv('data/models/xlsx/MMRNHep_rxns.csv')

In [144]:
for index in MMRNHep_rxns.ID:
    if index not in list(MMRN_rxns['RetiredID']):
        print(index)

MMRN_Lipid_pool
MMRN_Protein_pool
MMRN_Nucleotide_pool
MMRN_glycogen
MMRN_Biomass
HMR_biomass_Renalcancer(with ATP)
EXC_OUT_Biomass
EXC_OUT_Lipid_pool_biomass
EXC_OUT_Protein_pool_biomass
EXC_OUT_Nucleotide_pool_biomass
EXC_OUT_cofactors_vitamins
EXC_OUT_glycogen


# COMPARISON WITH MMRN-HEP

In [72]:
MMRN_rxns_revised = pd.read_excel('data/models/xlsx/MMRN_revised.xlsx',sheet_name='RXNS',index_col='ID')
MMRNHep_rxns = pd.read_excel('data/models/xlsx/genericLiverWD.xlsx',sheet_name='RXNS',index_col='ID')

In [77]:
select = []
for index in MMRNHep_rxns.index:
    if index not in list(MMRN_rxns_revised['RetiredID']):
        if 'EXC' not in index:
            select.append(index)
        else:
            select.append(index)
df = MMRNHep_rxns.loc[select]
df.to_csv('data/models/xlsx/toAdd.csv')

In [74]:
MMRN_mets_revised = pd.read_excel('data/models/xlsx/MMRN_revised.xlsx',sheet_name='METS',index_col='ID')
MMRNHep_mets = pd.read_excel('data/models/xlsx/genericLiverWD.xlsx',sheet_name='METS',index_col='ID')

In [75]:
select = []
for index in MMRNHep_mets.index:
    if index not in list(MMRN_mets_revised.index):
        if 'EXC' not in index:
            select.append(index)
df = MMRNHep_mets.loc[select]
df.to_csv('data/models/xlsx/toAddMets.csv')

In [78]:
MMRN_mets_revised = pd.read_excel('data/models/xlsx/MMRN_revised.xlsx',sheet_name='GENES',index_col='NAME')
MMRNHep_mets = pd.read_excel('data/models/xlsx/genericLiverWD.xlsx',sheet_name='GENES',index_col='NAME')

In [79]:
select = []
for index in MMRNHep_mets.index:
    if index not in list(MMRN_mets_revised.index):
        if 'EXC' not in index:
            select.append(index)
df = MMRNHep_mets.loc[select]
df

,#,MIRIAM,SHORT NAME,COMPARTMENT
NAME,,,,
ENSMUSG00000029199,NaN,NaN,NaN,NaN
ENSMUSG00000042460,NaN,NaN,NaN,NaN
ENSMUSG00000048970,NaN,NaN,NaN,NaN
ENSMUSG00000046794,NaN,NaN,NaN,NaN
